In [ ]:
# ---------------------------------- Libraries ----------------------------------
!pip install -U bitsandbytes
!pip install -q transformers accelerate bitsandbytes torch pypdf gradio
!pip install --upgrade transformers



In [ ]:
!pip install --upgrade transformers
import os
os.kill(os.getpid(), 9)  # force restarts the Colab runtime


In [ ]:
# ---------------------------------- Imports ----------------------------------
import gc
from google.colab import userdata
import gradio as gr
from huggingface_hub import login, notebook_login
from IPython.display import display, Markdown
import os
from pathlib import Path
import pypdf
import requests
import torch
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline



In [ ]:
# ---------------------------------- HuggingFace Connection ----------------------------------
hf_token = os.environ.get('HF_TOKEN') or userdata.get('HF_TOKEN')

if hf_token:
    login(token=hf_token)
    print("HuggingFace login successful.")
else:
    print("HuggingFace token not found. Please set the HF_TOKEN environment variable or store it in Colab secrets.")

In [ ]:
# ---------------------------------- Use GPU ----------------------------------
if torch.cuda.is_available():
    print(f"GPU detected: {torch.cuda.get_device_name(0)}")
    # Set default device to GPU
    torch.set_default_device("cuda")
    print("PyTorch default device set to CUDA (GPU).")
else:
    print("WARNING: No GPU detected. Running on CPU")

In [ ]:
# ---------------------------------- Functions ----------------------------------
def print_markdown(text):
    """Displays text as Markdown in Colab/Jupyter."""
    display(Markdown(text))



## transformers - A Python library that provides a standardized way to download, load, and use models from the Hub with just a few lines of code. Key classes:
- **pipeline()** - A high-level, easy-to-use abstraction for common tasks (like text generation, summarization). Great for quick tests and beginners.
- **AutoTokenizer** - Automatically downloads the correct "tokenizer" for a model. A tokenizer converts human-readable text into numerical IDs the model understands.
- **AutoModelFor...** - Automatically downloads the correct model architecture and pre-trained weights (e.g., AutoModelForCausalLM for text generation models like GPT, Llama, Gemma).
- Other Libraries - HF also develops libraries like accelerate (for efficient loading/distributed training), datasets (for handling datasets), and evaluate (for model evaluation metrics).



In [ ]:
# ---------------------------------- HuggingFace Open Source LLM Pipeline ----------------------------------
# Load a sentiment classifier model on financial news data
# https://huggingface.co/ProsusAI/finbert
pipe = pipeline(model = "ProsusAI/finbert")
pipe("Apple lost 10 Million dollars today due to US tarrifs")

The finbert model classified the string of text as negative with a high rating.
[{'label': 'negative', 'score': 0.9706032276153564}]

In [ ]:
# Load tokenizer for GPT-2
tokenizer = AutoTokenizer.from_pretrained("gpt2")
# Encode text to token IDs
tokens = tokenizer("Hello everyone and welcome to LLM and AI Agents Bootcamp")
print(f"gpt2 tokenizer:         {tokens}")
print(f"{tokens['input_ids']}")

# Load tokenizer for bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# Encode text to token IDs
tokens = tokenizer("Hello everyone and welcome to LLM and AI Agents Bootcamp")
print(f"bert-base-uncased:     {tokens}")
print(f"{tokens['input_ids']}")

# Load tokenizer for facebook/opt-125m
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")
# Encode text to token IDs
tokens = tokenizer("Hello everyone and welcome to LLM and AI Agents Bootcamp")
print(f"facebook/opt-125m:     {tokens}")
print(f"{tokens['input_ids']}")

gpt2 tokenizer:         {'input_ids': [15496, 2506, 290, 7062, 284, 27140, 44, 290, 9552, 28295, 18892, 16544], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

[15496, 2506, 290, 7062, 284, 27140, 44, 290, 9552, 28295, 18892, 16544]

bert-base-uncased:     {'input_ids': [101, 7592, 3071, 1998, 6160, 2000, 2222, 2213, 1998, 9932, 6074, 9573, 26468, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

[101, 7592, 3071, 1998, 6160, 2000, 2222, 2213, 1998, 9932, 6074, 9573, 26468, 102]

facebook/opt-125m:     {'input_ids': [2, 31414, 961, 8, 2814, 7, 30536, 448, 8, 4687, 24854, 21524, 21669], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

[2, 31414, 961, 8, 2814, 7, 30536, 448, 8, 4687, 24854, 21524, 21669]


Different tokenizers are better at different things.  Breaking up the tokens into less tokens that are longer makes the LLM respond faster because it has less tokens to process.

## AutoModelForCausalLM - HuggingFace Transformers Library
`AutoModelForCausalLM` - A Hugging Face class that automatically loads a pretrained model for causal (left-to-right) language modeling, such as GPT, LLaMA, or Gemma.
Key Steps:
- Choose a Model ID:
 Example - `google/gemma-2b-it` or `microsoft/Phi-3-mini-4k-instruct`
- Load the Tokenizer - `Use AutoTokenizer.from_pretrained(model_id)` to get the specific tokenizer for that model.
- Load the Model - Use `AutoModelForCausalLM.from_pretrained(...)` with crucial arguments:
  - `model_id` - The identifier.
  - `torch_dtype=torch.float16` (or `bfloat16`) - Loads the model using 16-bit floating point numbers instead of 32-bit, saving memory.
  - `load_in_4bit=True` or `load_in_8bit=True` - This is quantization via bitsandbytes. It further reduces memory by representing model weights with fewer bits (4 or 8 instead of 16/32). Essential for free Colab! 4-bit saves more memory but might have a tiny impact on quality compared to 8-bit.
  - `device_map="auto"` - Tells accelerate to automatically figure out how to spread the model across available devices (primarily the GPU in our case).
 - Combine Tokenizer and Model (Optional but common): Using the pipeline function is often simpler for basic text generation. It handles tokenization, model inference, and decoding back to text for you.




In [ ]:
# ---------------------------------- HuggingFace Open Source LLM ----------------------------------
# Load the model
# model_id = "unsloth/gemma-3-4b-it-GGUF"
model_id = "Qwen/Qwen2.5-3B-Instruct"
# model_id = "unsloth/Llama-3.2-3B-Instruct"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code = True)

# Create BitsAndBytesConfig for 4-bit quantization
quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                         bnb_4bit_compute_dtype = torch.float16,  # or torch.bfloat16 if available
                                         bnb_4bit_quant_type = "nf4",             # normal float 4 quantization
                                         bnb_4bit_use_double_quant = True         # use nested quantization for more efficient memory usage
                                         )

# Load the model with the quantization config
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config = quantization_config,
                                             device_map = "auto",
                                             trust_remote_code = True)

In [ ]:
# Let's define a prompt
prompt = "Explain how Electric Vehicles work in a funny way!"

In [ ]:
# ---------------------------------- Method 1 - Generate Response using `generate()` ----------------------------------
# Method 1 is used when the end user wants to tweak how they're interacting with the LLM more

# Let's encode the input first
inputs = tokenizer(prompt, return_tensors = "pt")

outputs = model.generate(**inputs, max_new_tokens = 1000)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print_markdown(response)

Explain how Electric Vehicles work in a funny way! Imagine that each battery in an electric vehicle is like a bunch of balloons, and the motor is like a super-strong helium blower. When you plug in the vehicle, it's like inflating all those balloons at once, making the car vibrate and dance around like a balloon animal! The more balloons (or batteries) you have, the bigger and more bouncy the car becomes. And when you use the brakes, it's like squeezing the balloons to make them deflate slowly, giving the car a fun, wiggly ride home. So, every time you drive, it's like a giant, floating party balloon going for a spin! But don't worry, it's all powered by clean, quiet, and very funny helium gas instead of noisy, smelly gasoline. Isn't that a silly yet creative way to think about electric vehicles?

In [ ]:
# ---------------------------------- Method 2 - Generate Response using `pipeline()` ----------------------------------
# Method 2 is used more when the end user wants a simple to use wrapper with less tweaking of the interaction with the LLM

# The pipeline wraps tokenization, generation, and decoding
pipe = pipeline("text-generation",
                model = model,
                tokenizer = tokenizer,
                torch_dtype = "auto", # Match model dtype
                device_map = "auto" # Ensure pipeline uses the same device mapping
                )


outputs = pipe(prompt,
               max_new_tokens = 1000, # max_new_tokens limits the length of the generated response.
               temperature = 1, # temperature controls randomness (lower = more focused).
               )

# Print the generated text
print_markdown(outputs[0]['generated_text'])

Explain how Electric Vehicles work in a funny way! Imagine you have a bunch of toys, like little cars or trucks, and you want to make them run on electricity instead of gasoline. To do this, you need a special power plant, which we'll call the "charger plant." This plant has magical glowing wires that light up when it's ready to give juice to your toy vehicles.

Now, every vehicle needs a battery box, kind of like a big container for holding electricity. When you plug one of these vehicles into the charger plant, the glowing wires start shooting out tiny energy beams towards the vehicle's battery box. The beams travel through invisible super-speedy tunnels inside the vehicle, just like how you might imagine a superhero flying through the air!

As the glowing beams reach the battery box, they turn the battery's magic switch (which is actually a small button) on. The vehicle starts humming and whirring, much like when you press a toy car's starting button but much faster.

When the vehicle runs out of energy, its magic switch (button) turns off automatically, and the vehicle slows down until it stops. At this point, the glowing wires from the charger plant come back to the charger plant itself. You don't even need to think about it; it's all automatic!

The best part? When the vehicle runs out of energy, it automatically tells the charger plant to refill its batteries with a new batch of glowing energy beams. It's like the vehicle is saying, "I'm low on juice! Give me some more!" So you don't even need to remember to change the batteries – the vehicle takes care of it all by itself.

In the end, you have a bunch of fun, shiny electric vehicles that are easy to operate and take care of, making driving and having fun a breeze! Isn't that silly and exciting?


In [ ]:
# ---------------------------------- Extracting pdf Text ----------------------------------
# --- Get the PDF File ---
pdf_url = "https://s206.q4cdn.com/479360582/files/doc_financials/2025/q1/2025q1-alphabet-earnings-release.pdf"
pdf_filename = "google_earning_transcript.pdf"
pdf_path = Path(pdf_filename)

# Download the file if it doesn't exist
if not pdf_path.exists():
    response = requests.get(pdf_url)
    response.raise_for_status()  # Check for download errors
    pdf_path.write_bytes(response.content)
    print(f"PDF downloaded successfully to {pdf_path}")
else:
    print(f"PDF file already exists at {pdf_path}")


# --- Read Text from PDF using pypdf ---
pdf_text = ""

print(f"Reading text from {pdf_path}...")
reader = pypdf.PdfReader(pdf_path)
num_pages = len(reader.pages)
print(f"PDF has {num_pages} pages.")

# Extract text from each page
all_pages_text = []
for i, page in enumerate(reader.pages):

    page_text = page.extract_text()
    if page_text:  # Only add if text extraction was successful
        all_pages_text.append(page_text)
    # print(f"Read page {i+1}/{num_pages}") # Uncomment for progress

# Join the text from all pages
pdf_text = "\n".join(all_pages_text)
print(f"Successfully extracted text. Total characters: {len(pdf_text)}")

# Display a small snippet of the PDF
print("\n--- Snippet of Extracted Text ---")
print_markdown(f"{pdf_text[:1000]}")

In [ ]:
# ---------------------------------- Pdf Reading Function ----------------------------------
# Define a limit for the context length to avoid overwhelming the model
MAX_CONTEXT_CHARS = 6000

def answer_question_from_pdf(document_text, question, llm_pipeline):
    """
    Answers a question based on the provided document text using the loaded LLM pipeline.

    Args:
        document_text (str): The text extracted from the PDF.
        question (str): The user's question.
        llm_pipeline (transformers.pipeline): The initialized text-generation pipeline.

    Returns:
        str: The model's generated answer.
    """
    # Truncate context if necessary
    if len(document_text) > MAX_CONTEXT_CHARS:
        print(f"Warning: Document text ({len(document_text)} chars) exceeds limit ({MAX_CONTEXT_CHARS} chars). Truncating.")
        context = document_text[:MAX_CONTEXT_CHARS] + "..."
    else:
        context = document_text

    # Prompt Template
    prompt_template = f"""<|system|>
    You are an AI assistant. Answer the following question based *only* on the provided document text. If the answer is not found in the document, say "The document does not contain information on this topic." Do not use any prior knowledge.

    Document Text:
    ---
    {context}
    ---
    <|end|>
    <|user|>
    Question: {question}<|end|>
    <|assistant|>
    Answer:"""

    print(f"\n--- Generating Answer for: '{question}' ---")

    # Run Inference on the chosen model
    outputs = llm_pipeline(prompt_template,
                           max_new_tokens = 500,  # Limit answer length
                           do_sample = True,
                           temperature = 0.2,   # Lower temperature for more factual Q&A
                           top_p = 0.9)

    # Extract the answer
    full_generated_text = outputs[0]['generated_text']
    answer_start_index = full_generated_text.find("Answer:") + len("Answer:")
    raw_answer = full_generated_text[answer_start_index:].strip()

    # Sometimes the model might still include parts of the prompt or trail off.
    # Basic cleanup: Find the end-of-sequence token if possible, or just return raw.
    # Phi-3 uses <|end|> or <|im_end|>
    end_token = "<|end|>"
    if end_token in raw_answer:
            raw_answer = raw_answer.split(end_token)[0]

    print("--- Generation Complete ---")
    return raw_answer

In [ ]:
# Test the function
test_question = "What is this document about?"
generated_answer = answer_question_from_pdf(pdf_text, test_question, pipe)

print("\nTest Question:")
print_markdown(f"**Q:** {test_question}")
print("\nGenerated Answer:")
print_markdown(f"**A:** {generated_answer}")

Test Question:

Q: What is this document about?


Generated Answer:

A: This document is about Alphabet Inc.'s (formerly Google) financial results for the first quarter of 2025, including revenues, operating income, net income, and other financial metrics. It also includes details on the company's segment operating results, dividend program, and stock repurchase authorization. The document provides a comprehensive overview of Alphabet's financial performance and strategic initiatives for the quarter. <|end>|


In [ ]:
# ---------------------------------- Gradio App ----------------------------------
available_models = {
    "Llama 3.2": "unsloth/Llama-3.2-3B-Instruct",
    "Microsoft Phi-4 Mini": "microsoft/Phi-4-mini-instruct",
    "Google Gemma 3": "unsloth/gemma-3-4b-it-GGUF"
    }

# --- Global State (or use gr.State in Blocks) ---
# To keep track of the currently loaded model/pipeline
current_model_id = None
current_pipeline = None
print(f"Models available for selection: {list(available_models.keys())}")


# Define a function to Load/Switch Models
def load_llm_model(model_name):
    """Loads the selected LLM, unloading the previous one."""
    global current_model_id, current_pipeline, tokenizer, model

    new_model_id = available_models.get(model_name)
    if not new_model_id:
        return "Invalid model selected.", None  # Return error message and None pipeline

    if new_model_id == current_model_id and current_pipeline is not None:
        print(f"Model {model_name} is already loaded.")
        # Indicate success but don't reload
        return f"{model_name} already loaded.", current_pipeline

    print(f"Switching to model: {model_name} ({new_model_id})...")

    # Unload previous model (important for memory)
    # Clear variables and run garbage collection
    current_pipeline = None
    if "model" in locals():
        del model
    if "tokenizer" in locals():
        del tokenizer
    if "pipe" in locals():
        del pipe
    torch.cuda.empty_cache()  # Clear GPU memory cache

    gc.collect()
    print("Previous model unloaded (if any).")

    # --- Load the new model ---
    loading_message = f"Loading {model_name}..."
    try:
        # Load Tokenizer
        tokenizer = AutoTokenizer.from_pretrained(new_model_id, trust_remote_code = True)

        # Load Model (Quantized)
        model = AutoModelForCausalLM.from_pretrained(new_model_id,
                                                     torch_dtype = "auto",  # "torch.float16", # Or bfloat16 if available
                                                     load_in_4bit = True,
                                                     device_map = "auto",
                                                     trust_remote_code = True)

        # Create Pipeline
        loaded_pipeline = pipeline(
            "text-generation", model = model, tokenizer = tokenizer, torch_dtype = "auto", device_map = "auto")

        print(f"Model {model_name} loaded successfully!")
        current_model_id = new_model_id
        current_pipeline = loaded_pipeline  # Update global state
        # Use locals() or return values with gr.State for better Gradio practice
        return f"{model_name} loaded successfully!", loaded_pipeline  # Status message and the pipeline object

    except Exception as e:
        print(f"Error loading model {model_name}: {e}")
        current_model_id = None
        current_pipeline = None
        return f"Error loading {model_name}: {e}", None  # Error message and None pipeline

In [ ]:
# --- Function to handle Q&A Submission ---
def handle_submit(question):
    """Handles the user submitting a question."""
    if not current_pipeline:
        return "Error: No model is currently loaded. Please select a model."
    if not pdf_text:
        return "Error: PDF text is not loaded. Please run Section 4."
    if not question:
        return "Please enter a question."

    print(f"Handling submission for question: '{question}' using {current_model_id}")
    # Call the Q&A function defined in Section 5
    answer = answer_question_from_pdf(pdf_text, question, current_pipeline)
    return answer


In [ ]:
# --- Build Gradio Interface using Blocks ---
print("Building Gradio interface...")
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        f"""
    # PDF Q&A Bot Using Hugging Face Open-Source Models
    Ask questions about the document ('{pdf_filename}' if loaded, {len(pdf_text)} chars).
    Select an open-source LLM to answer your question.
    **Note:** Switching models takes time as the new model needs to be downloaded and loaded into the GPU.
    """
    )

    # Store the pipeline in Gradio state for better practice (optional for this simple version)
    # llm_pipeline_state = gr.State(None)

    with gr.Row():
        model_dropdown = gr.Dropdown(
            choices=list(available_models.keys()),
            label="🤖 Select LLM Model",
            value=list(available_models.keys())[0],  # Default to the first model
        )
        status_textbox = gr.Textbox(label="Model Status", interactive=False)

    question_textbox = gr.Textbox(
        label="❓ Your Question", lines=2, placeholder="Enter your question about the document here..."
    )
    submit_button = gr.Button("Submit Question", variant="primary")
    answer_textbox = gr.Textbox(label="💡 Answer", lines=5, interactive=False)

    # --- Event Handlers ---
    # When the dropdown changes, load the selected model
    model_dropdown.change(
        fn = load_llm_model,
        inputs = [model_dropdown],
        outputs = [status_textbox],  # Update status text. Ideally also update a gr.State for the pipeline
        # outputs=[status_textbox, llm_pipeline_state] # If using gr.State
    )

    # When the button is clicked, call the submit handler
    submit_button.click(
        fn = handle_submit,
        inputs = [question_textbox],
        outputs = [answer_textbox],
        # inputs=[question_textbox, llm_pipeline_state], # Pass state if using it
    )

    # --- Initial Model Load ---
    # Easier: Manually load first model *before* launching Gradio for simplicity here
    initial_model_name = list(available_models.keys())[0]
    print(f"Performing initial load of default model: {initial_model_name}...")
    status, _ = load_llm_model(initial_model_name)
    status_textbox.value = status  # Set initial status
    print("Initial load complete.")


# --- Launch the Gradio App ---
print("Launching Gradio demo...")
demo.launch(debug=True)  # debug=True provides more detailed logs